In [12]:
import os, shutil
import skimage as ski
import numpy as np

from os import walk, getcwd
from skimage import io
from skimage import filters
from skimage import morphology
from skimage import measure
from matplotlib import pyplot as plt
%matplotlib inline


# Retorna todos los archivos de un directorio dado
def ls(ruta):  
    return next(walk(ruta))[2]

In [13]:
#Ejecutar sólo una vez, y si el dataset: data_augmentation_dataset no existe

#Definimos origen y destino de datasets y nombres
original_dataset_dir = 'split_dataset' #origen
base_dir = 'cropped_dataset'           #destino
os.mkdir(base_dir) #Creamos directorio dataset destino

#Creamos los tres directorios train, validation y test dentro de data_augmentation_dataset
train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

#Creamos las carpetas de las 4 clases dentro de la carpeta train
train_atypical_dir = os.path.join(train_dir, 'Atypical_Lymphocyte')
os.mkdir(train_atypical_dir)
train_blast_dir = os.path.join(train_dir, 'Blast')
os.mkdir(train_blast_dir)
train_lymphocyte_dir = os.path.join(train_dir,'Lymphocyte')
os.mkdir(train_lymphocyte_dir)
train_variant_dir = os.path.join(train_dir,'Variant_Lymphocyte')
os.mkdir(train_variant_dir)

#Creamos las carpetas de las 4 clases dentro de la carpeta validation
validation_atypical_dir = os.path.join(validation_dir, 'Atypical_Lymphocyte')
os.mkdir(validation_atypical_dir)
validation_blast_dir = os.path.join(validation_dir, 'Blast')
os.mkdir(validation_blast_dir)
validation_lymphocyte_dir = os.path.join(validation_dir,'Lymphocyte')
os.mkdir(validation_lymphocyte_dir)
validation_variant_dir = os.path.join(validation_dir,'Variant_Lymphocyte')
os.mkdir(validation_variant_dir)

#Creamos las carpetas de las 4 clases dentro de la carpeta test
test_atypical_dir = os.path.join(test_dir, 'Atypical_Lymphocyte')
os.mkdir(test_atypical_dir)
test_blast_dir = os.path.join(test_dir, 'Blast')
os.mkdir(test_blast_dir)
test_lymphocyte_dir = os.path.join(test_dir,'Lymphocyte')
os.mkdir(test_lymphocyte_dir)
test_variant_dir = os.path.join(test_dir,'Variant_Lymphocyte')
os.mkdir(test_variant_dir)


In [14]:
def crop(I, width, height, c):
    Fs = int( int(c[0]) - 0.5 * height )
    Coli = int( int(c[1]) - 0.5 * width )
    #Cold = int( 2400 - int(c[1])-0.5*width )
    #Fi = int( 1800 - int(c[0]) - 0.5*height )
    Cold = int( 600 - int(c[1])-0.5 * width )
    Fi = int( 450 - int(c[0]) - 0.5 * height )
    
    print(Fs,Fi,Coli,Cold)
    
    # Taking in account a cell close to the borders
    if Fs < 0:
        Fi +=Fs
        Fs = 0        
    if Fi < 0:
        Fs += Fi
        Fi = -1801
    if Coli < 0:
        Cold += Coli
        Coli = 0        
    if Cold < 0:
        Coli += Cold
        Cold = -2401        
          
    #print(Fs,-Fi,Coli,-Cold)
        
    return I[Fs:-Fi, Coli:-Cold, :]

In [15]:
def crop_dataset (I):
    
    I = ski.img_as_float(I) # Convert the image to float (64 bits)
    G = I[:,:,1]
    #plt.imshow(G, cmap ='gray') #It is better specify the colormap "cmap"
    
    from skimage import filters
    G = filters.gaussian(G, sigma = 4)
    #plt.imshow(G, cmap = 'gray')
     
    th = filters.threshold_minimum(G) # threshold by the isodata or minimum method of the green component 
        
    mask = G < th # binary mas of the foreground
    #plt.imshow(mask, cmap = 'gray')
    
    from skimage import morphology
    mask2 = morphology.remove_small_objects(mask, 4000) #4000
    #plt.imshow(mask2, cmap = 'gray')
    
    from skimage import measure
    mask_label = measure.label(mask2, background = 0) #Labeling the mask
    props = measure.regionprops(mask_label) # generate a list that contains the properties (measures) for each region
    
    cc = [prop.centroid for prop in props]
     
    for c in cc:
        I = ski.img_as_ubyte(I)
        #I2 = crop(1220,1230, c)
        I2 = crop(I, 2500,2510, c)
        #plt.imshow(I2)
    
    z = [int(c) for c in cc[0]] ####Añadimos z
    
    areas = [prop.area for prop in props]
    #print(areas)
    
    c = [int(i) for i in cc[0]]
    
    #plt.imshow(G[300:-200,400:-600], 'gray')
    #plt.imshow(G[100:-1,250:-100], 'gray')
    
    zz = int(np.round(100.8))  #####Añadimos zz
    
    I = ski.img_as_ubyte(I)
    I2 = crop(I, 1500,1350, cc[0])
    #plt.imshow(I2)
 
    return I2

In [16]:
def init_crop(orig_dir, dest_dir, archives):

    for archive in archives:
        dt_dir = os.path.join(orig_dir, archive) #origen + nombre_imagen
        #print(dt_dir)
        I = io.imread(dt_dir)
        
        I2 = crop_dataset(I)     
               
        destino = os.path.join(dest_dir, archive) #destino + nombre_imagen
        io.imsave(destino,I2)
       

    return I2

In [25]:
def init_crop_atypical(folder, orig_dir, dest_dir, archives):
    i = 1
    for archive in archives:
        dt_dir = os.path.join(orig_dir, archive) #origen + nombre_imagen
        #print(dt_dir)
        I = io.imread(dt_dir)
        
        if folder == 'train' : 
            if i == 161:# or i == 161 i == 377 or i == 378:  
                I2 = I
            else:
                I2 = crop_dataset(I)   
                
        if folder == 'validation':
            if i == 286: # or i == 377 or i == 378:  
                I2 = I
            else:
                I2 = crop_dataset(I)   
            
        if folder == 'test':
            if i == 227:  
                I2 = I
            else:
                I2 = crop_dataset(I)
            
        destino = os.path.join(dest_dir, archive) #destino + nombre_imagen
        io.imsave(destino,I2)
        i += 1 
    return I2

In [59]:
def init_crop_blast(folder, orig_dir, dest_dir, archives):
    i = 1
    for archive in archives:
        dt_dir = os.path.join(orig_dir, archive) #origen + nombre_imagen
        #print(dt_dir)
        I = io.imread(dt_dir)
        
        if folder == 'train' : 
            if i == 4 or i == 12 or i == 34 or i == 294 or i == 366:  
                I2 = I
            else:
                I2 = crop_dataset(I)                  

        if folder == 'validation':
            if i == 63 or i == 76: # or i == 377 or i == 378:  
                I2 = I
            else:
                I2 = crop_dataset(I)   
            
        if folder == 'test':
            if i == 293: # or i == 260 or i == 485:  
                I2 = I
            else:
                I2 = crop_dataset(I)
            
         
        destino = os.path.join(dest_dir, archive) #destino + nombre_imagen
        io.imsave(destino,I2)
        i += 1
    return I2

In [19]:
def read_images(folder, subfolder):
    
    #origen
    orig_dir =  os.path.join(original_dataset_dir, folder, subfolder)
    archives = (ls(ruta = orig_dir))
    
    #destino
    dest_dir = os.path.join(base_dir, folder, subfolder)
    
    return (orig_dir, dest_dir, archives)



In [28]:
#Leemos las imágenes originales y las copiamos en directorios train, validation y test recortadas
#SI
folders= ['train', 'validation', 'test']

for folder in folders: 
    subfolder = 'Atypical_Lymphocyte'      
    [orig_dir, dest_dir, archives] = read_images(folder, subfolder)
    I = init_crop_atypical(folder, orig_dir, dest_dir, archives) #Recorte y guardamos en disco

for folder in folders: 
    subfolder = 'Blast'
    [orig_dir, dest_dir, archives] = read_images(folder, subfolder)
    init_crop_blast(folder, orig_dir, dest_dir, archives) #Recorte y guardamos en disco

/usr/local/lib/python3.5/dist-packages/skimage/morphology/misc.py:122: UserWarning: Only one label was provided to `remove_small_objects`. Did you mean to use a boolean array?
  warn("Only one label was provided to `remove_small_objects`. "
/usr/local/lib/python3.5/dist-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


-737 -1323 -644 -1256
-157 -743 -144 -756
-704 -1356 -563 -1337
-502 -1558 -96 -1804
-124 -776 -63 -837
-719 -1341 -600 -1300
-139 -761 -100 -800
-793 -1267 -550 -1350
-213 -687 -50 -850
-768 -1292 -509 -1391
-188 -712 -9 -891
-744 -1316 -656 -1244
-164 -736 -156 -744
-759 -1301 -681 -1219
-179 -721 -181 -719
-847 -1213 -593 -1307
-267 -633 -93 -807
-841 -1219 -693 -1207
-261 -639 -193 -707
-749 -1311 -566 -1334
-169 -731 -66 -834
-688 -1372 -680 -1220
-108 -792 -180 -720
-636 -1424 -466 -1434
-56 -844 34 -934
-795 -1265 -499 -1401
-215 -685 1 -901
-800 -1260 -660 -1240
-220 -680 -160 -740
-842 -1218 -694 -1206
-262 -638 -194 -706
-703 -1357 -633 -1267
-123 -777 -133 -767
-680 -1380 -307 -1593
-100 -800 193 -1093
-732 -1328 -758 -1142
-152 -748 -258 -642
-738 -1322 -618 -1282
-158 -742 -118 -782
-844 -1216 -498 -1402
-837 -1223 -297 -1603
-264 -636 2 -902
-869 -1191 -656 -1244
-289 -611 -156 -744
-788 -1272 -647 -1253
-811 -1249 -89 -1811
-208 -692 -147 -753
-926 -1134 -770 -1130
-829 

-619 -1441 -492 -1408
-39 -861 8 -908
-725 -1335 -465 -1435
-664 -1396 -662 -1238
-443 -1617 -1212 -688
-145 -755 35 -935
-829 -1231 -715 -1185
-249 -651 -215 -685
-762 -1298 -494 -1406
-182 -718 6 -906
-778 -1282 -701 -1199
-198 -702 -201 -699
-722 -1338 -540 -1360
-142 -758 -40 -860
-796 -1264 -420 -1480
-216 -684 80 -980
-796 -1264 -557 -1343
-216 -684 -57 -843
-742 -1318 -640 -1260
-162 -738 -140 -760
-730 -1330 -697 -1203
-150 -750 -197 -703
-1116 -944 -602 -1298
-825 -1235 -561 -1339
-536 -364 -102 -798
-830 -1230 -660 -1240
-250 -650 -160 -740
-923 -1137 -820 -1080
-343 -557 -320 -580
-660 -1400 -562 -1338
-80 -820 -62 -838
-789 -1271 -886 -1014
-209 -691 -386 -514
-844 -1216 -726 -1174
-264 -636 -226 -674
-736 -1324 -625 -1275
-156 -744 -125 -775
-881 -1179 -440 -1460
-301 -599 60 -960
-762 -1298 -572 -1328
-182 -718 -72 -828
-665 -1395 -574 -1326
-85 -815 -74 -826
-769 -1291 -619 -1281
-189 -711 -119 -781
-857 -1203 -992 -908
-690 -1370 -707 -1193
-670 -1390 -952 -948
-508 -15

In [61]:
dest_dir = os.path.join(base_dir, folder, subfolder)
archives = (ls(ruta = dest_dir))
print(dest_dir)
len(archives)
#plt.imshow(I)

cropped_dataset/test/Blast


300

In [62]:
#Leemos las imágenes originales y las copiamos en directorios train, validation y test recortadas
#SI
folders= ['train', 'validation', 'test']

for folder in folders: 
   
    subfolder = 'Lymphocyte'  
    [orig_dir, dest_dir, archives] = read_images(folder, subfolder)
    I = init_crop(orig_dir, dest_dir, archives) #Recorte y guardamos en disco
    
    subfolder = 'Variant_Lymphocyte'  
    [orig_dir, dest_dir, archives] = read_images(folder, subfolder)
    I = init_crop(orig_dir, dest_dir, archives) #Recorte y guardamos en disco

/usr/local/lib/python3.5/dist-packages/skimage/morphology/misc.py:122: UserWarning: Only one label was provided to `remove_small_objects`. Did you mean to use a boolean array?
  warn("Only one label was provided to `remove_small_objects`. "
/usr/local/lib/python3.5/dist-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


-825 -1235 -881 -1019
-245 -655 -381 -519
-900 -1160 -460 -1440
-320 -580 40 -940
-925 -1135 -618 -1282
-345 -555 -118 -782
-737 -1323 -604 -1296
-157 -743 -104 -796
-719 -1341 -754 -1146
-139 -761 -254 -646
-689 -1371 -518 -1382
-109 -791 -18 -882
-629 -1431 -666 -1234
-49 -851 -166 -734
-754 -1306 -604 -1296
-174 -726 -104 -796
-784 -1276 -618 -1282
-204 -696 -118 -782
-917 -1143 -538 -1362
-337 -563 -38 -862
-704 -1356 -623 -1277
-124 -776 -123 -777
-1059 -1001 -733 -1167
-479 -421 -233 -667
-805 -1255 -716 -1184
-225 -675 -216 -684
-707 -1353 -655 -1245
-127 -773 -155 -745
-753 -1307 -512 -1388
-173 -727 -12 -888
-783 -1277 -686 -1214
-203 -697 -186 -714
-844 -1216 -656 -1244
-264 -636 -156 -744
-689 -1371 -446 -1454
-109 -791 54 -954
-653 -1407 -527 -1373
-73 -827 -27 -873
-767 -1293 -303 -1597
-187 -713 197 -1097
-727 -1333 -738 -1162
-147 -753 -238 -662
-788 -1272 -513 -1387
-208 -692 -13 -887
-809 -1251 -561 -1339
-229 -671 -61 -839
-864 -1196 -651 -1249
-284 -616 -151 -749
-58

-822 -1238 -435 -1465
-242 -658 65 -965
-796 -1264 -427 -1473
-216 -684 73 -973
-714 -1346 -512 -1388
-134 -766 -12 -888
-1022 -1038 -625 -1275
-442 -458 -125 -775
-802 -1258 -601 -1299
-222 -678 -101 -799
-701 -1359 -660 -1240
-121 -779 -160 -740
-852 -1208 -733 -1167
-272 -628 -233 -667
-749 -1311 -244 -1656
-169 -731 256 -1156
-798 -1262 -595 -1305
-218 -682 -95 -805
-870 -1190 -865 -1035
-290 -610 -365 -535
-759 -1301 -641 -1259
-179 -721 -141 -759
-710 -1350 -679 -1221
-130 -770 -179 -721
-966 -1094 -737 -1163
-739 -1321 -601 -1299
-386 -514 -237 -663
-772 -1288 -786 -1114
-192 -708 -286 -614
-896 -1164 -528 -1372
-316 -584 -28 -872
-808 -1252 -695 -1205
-228 -672 -195 -705
-849 -1211 -848 -1052
-269 -631 -348 -552
-840 -1220 -578 -1322
-260 -640 -78 -822
-782 -1278 -661 -1239
-202 -698 -161 -739
-836 -1224 -782 -1118
-256 -644 -282 -618
-577 -1483 -347 -1553
3 -903 153 -1053
-745 -1315 -584 -1316
-165 -735 -84 -816
-784 -1276 -571 -1329
-204 -696 -71 -829
-844 -1216 -964 -936
-26

-720 -1340 -722 -1178
-140 -760 -222 -678
-757 -1303 -521 -1379
-177 -723 -21 -879
-574 -1486 -608 -1292
6 -906 -108 -792
-750 -1310 -553 -1347
-170 -730 -53 -847
-623 -1437 -657 -1243
-43 -857 -157 -743
-764 -1296 -823 -1077
-184 -716 -323 -577
-816 -1244 -386 -1514
-236 -664 114 -1014
-802 -1258 -845 -1055
-222 -678 -345 -555
-659 -1401 -546 -1354
-79 -821 -46 -854
-824 -1236 -715 -1185
-244 -656 -215 -685
-1006 -1054 -658 -1242
-426 -474 -158 -742
-900 -1160 -746 -1154
-320 -580 -246 -654
-701 -1359 -705 -1195
-121 -779 -205 -695
-712 -1348 -1098 -802
-595 -1465 -670 -1230
-132 -768 -598 -302
-1203 -857 -369 -1531
-1159 -901 -443 -1457
-759 -1301 -759 -1141
-623 -277 131 -1031
-946 -1114 -511 -1389
-366 -534 -11 -889
-771 -1289 -517 -1383
-191 -709 -17 -883
-757 -1303 -593 -1307
-177 -723 -93 -807
-811 -1249 -693 -1207
-231 -669 -193 -707
-1135 -925 -528 -1372
-555 -345 -28 -872
-574 -1486 -714 -1186
6 -906 -214 -686
-1016 -1044 -1067 -833
-686 -1374 -572 -1328
-436 -464 -567 -333
-

-918 -1142 -616 -1284
-338 -562 -116 -784
-704 -1356 -888 -1012
-124 -776 -388 -512
-957 -1103 -556 -1344
-377 -523 -56 -844
-754 -1306 -692 -1208
-174 -726 -192 -708
-790 -1270 -825 -1075
-210 -690 -325 -575
-712 -1348 -559 -1341
-132 -768 -59 -841
-757 -1303 -488 -1412
-177 -723 12 -912
-688 -1372 -674 -1226
-108 -792 -174 -726
-732 -1328 -515 -1385
-152 -748 -15 -885
-748 -1312 -496 -1404
-168 -732 4 -904
-695 -1365 -424 -1476
-115 -785 76 -976
-846 -1214 -593 -1307
-653 -1407 -91 -1809
-543 -1517 -110 -1790
-266 -634 -93 -807
-838 -1222 -708 -1192
-258 -642 -208 -692
-914 -1146 -756 -1144
-334 -566 -256 -644
-613 -1447 -597 -1303
-33 -867 -97 -803
-731 -1329 -728 -1172
-151 -749 -228 -672
-732 -1328 -542 -1358
-152 -748 -42 -858
-781 -1279 -627 -1273
-201 -699 -127 -773
-642 -1418 -507 -1393
-62 -838 -7 -893
-787 -1273 -469 -1431
-207 -693 31 -931
-875 -1185 -898 -1002
-295 -605 -398 -502
-701 -1359 -568 -1332
-121 -779 -68 -832
-719 -1341 -579 -1321
-139 -761 -79 -821
-786 -1274 -

...

-577 -1483 -331 -1569
3 -903 169 -1069
-816 -1244 -864 -1036
-236 -664 -364 -536
-972 -1088 -921 -979
-392 -508 -421 -479
-809 -1251 -585 -1315
-229 -671 -85 -815
-956 -1104 -728 -1172
-376 -524 -228 -672
-745 -1315 -419 -1481
-165 -735 81 -981
-852 -1208 -603 -1297
-272 -628 -103 -797
-684 -1376 -344 -1556
-104 -796 156 -1056
-839 -1221 -492 -1408
-259 -641 8 -908
-930 -1130 -620 -1280
-350 -550 -120 -780
-641 -1419 -106 -1794
-616 -1444 -497 -1403
-61 -839 394 -1294
-1064 -996 -870 -1030
-484 -416 -370 -530
-771 -1289 -768 -1132
-191 -709 -268 -632
-718 -1342 -711 -1189
-138 -762 -211 -689
-910 -1150 -535 -1365
-330 -570 -35 -865
-782 -1278 -323 -1577
-202 -698 177 -1077
-882 -1178 -485 -1415
-302 -598 15 -915
-717 -1343 -539 -1361
-137 -763 -39 -861
-804 -1256 -766 -1134
-224 -676 -266 -634
-706 -1354 -701 -1199
-126 -774 -201 -699
-855 -1205 -582 -1318
-275 -625 -82 -818
-782 -1278 -532 -1368
-661 -1399 -152 -1748
-202 -698 -32 -868
-796 -1264 -641 -1259
-216 -684 -141 -759
-651 -1

-768 -1292 -546 -1354
-188 -712 -46 -854
-791 -1269 -602 -1298
-211 -689 -102 -798
-789 -1271 -419 -1481
-209 -691 81 -981
-845 -1215 -461 -1439
-265 -635 39 -939
-677 -1383 -725 -1175
-97 -803 -225 -675
-1025 -1035 -504 -1396
-445 -455 -4 -896
-685 -1375 -409 -1491
-105 -795 91 -991
-733 -1327 -521 -1379
-153 -747 -21 -879
-652 -1408 -691 -1209
-72 -828 -191 -709
-784 -1276 -762 -1138
-204 -696 -262 -638
-526 -1534 -467 -1433
54 -954 33 -933
-794 -1266 -444 -1456
-214 -686 56 -956
-842 -1218 -531 -1369
-262 -638 -31 -869
-923 -1137 -562 -1338
-343 -557 -62 -838
-692 -1368 -430 -1470
-718 -1342 -1173 -727
-112 -788 70 -970
-711 -1349 -344 -1556
-131 -769 156 -1056
-721 -1339 -483 -1417
-141 -759 17 -917
-762 -1298 -582 -1318
-182 -718 -82 -818
-656 -1404 -494 -1406
-76 -824 6 -906
-734 -1326 -532 -1368
-154 -746 -32 -868
-800 -1260 -683 -1217
-220 -680 -183 -717
-739 -1321 -674 -1226
-159 -741 -174 -726
-705 -1355 -542 -1358
-125 -775 -42 -858
-658 -1402 -521 -1379
-406 -1654 -703 -119

-802 -1258 -744 -1156
-222 -678 -244 -656
-668 -1392 -507 -1393
-88 -812 -7 -893
-872 -1188 -652 -1248
-292 -608 -152 -748
-915 -1145 -542 -1358
-335 -565 -42 -858
-727 -1333 -643 -1257
-147 -753 -143 -757
-863 -1197 -289 -1611
-283 -617 211 -1111
-821 -1239 -747 -1153
-241 -659 -247 -653
-782 -1278 -524 -1376
-202 -698 -24 -876
-653 -1407 -527 -1373
-73 -827 -27 -873
-787 -1273 -683 -1217
-207 -693 -183 -717
-790 -1270 -787 -1113
-210 -690 -287 -613
-770 -1290 -749 -1151
-190 -710 -249 -651
-845 -1215 -599 -1301
-265 -635 -99 -801
-868 -1192 -690 -1210
-288 -612 -190 -710
-841 -1219 -590 -1310
-261 -639 -90 -810
-695 -1365 -540 -1360
-115 -785 -40 -860
-647 -1413 -787 -1113
-67 -833 -287 -613
-656 -1404 -699 -1201
-76 -824 -199 -701
-885 -1175 -702 -1198
-305 -595 -202 -698
-641 -1419 -422 -1478
-61 -839 78 -978
-1159 -901 -307 -1593
-769 -1291 -512 -1388
-579 -321 193 -1093
-801 -1259 -504 -1396
-221 -679 -4 -896
-787 -1273 -400 -1500
-207 -693 100 -1000
-774 -1286 -555 -1345
-194 -7

-734 -1326 -427 -1473
-154 -746 73 -973
-663 -1397 -625 -1275
-83 -817 -125 -775
-750 -1310 -748 -1152
-170 -730 -248 -652
-700 -1360 -585 -1315
-120 -780 -85 -815
-949 -1111 -785 -1115
-369 -531 -285 -615
-797 -1263 -740 -1160
-217 -683 -240 -660
-818 -1242 -731 -1169
-238 -662 -231 -669
-787 -1273 -771 -1129
-207 -693 -271 -629
-892 -1168 -624 -1276
-312 -588 -124 -776
-793 -1267 -324 -1576
-213 -687 176 -1076
-641 -1419 -665 -1235
-61 -839 -165 -735
-557 -1503 -591 -1309
-443 -1617 -909 -991
23 -923 -91 -809
-944 -1116 -365 -1535
-770 -1290 -488 -1412
-364 -536 135 -1035
-682 -1378 -835 -1065
-102 -798 -335 -565
-683 -1377 -356 -1544
-103 -797 144 -1044
-820 -1240 -516 -1384
-240 -660 -16 -884
-737 -1323 -699 -1201
-157 -743 -199 -701
-744 -1316 -1163 -737
-667 -1393 -723 -1177
-611 -1449 -973 -927
-164 -736 -663 -237
-787 -1273 -860 -1040
-207 -693 -360 -540
-612 -1448 -763 -1137
-32 -868 -263 -637
-874 -1186 -868 -1032
-740 -1320 -578 -1322
-294 -606 -368 -532
-854 -1206 -810 -109

-587 -1473 -374 -1526
-7 -893 126 -1026
-1015 -1045 -512 -1388
-435 -465 -12 -888
-667 -1393 -641 -1259
-87 -813 -141 -759
-721 -1339 -768 -1132
-141 -759 -268 -632
-646 -1414 -778 -1122
-66 -834 -278 -622
-857 -1203 -813 -1087
-277 -623 -313 -587
-739 -1321 -659 -1241
-159 -741 -159 -741
-791 -1269 -673 -1227
-211 -689 -173 -727
-631 -1429 -603 -1297
-51 -849 -103 -797
-620 -1440 -451 -1449
-40 -860 49 -949
-571 -1489 -314 -1586
9 -909 186 -1086
-778 -1282 -538 -1362
-198 -702 -38 -862
-768 -1292 -534 -1366
-188 -712 -34 -866
-707 -1353 -706 -1194
-127 -773 -206 -694
-895 -1165 -685 -1215
-315 -585 -185 -715
-685 -1375 -660 -1240
-105 -795 -160 -740
-763 -1297 -569 -1331
-183 -717 -69 -831
-741 -1319 -594 -1306
-161 -739 -94 -806
-795 -1265 -777 -1123
-215 -685 -277 -623
-822 -1238 -449 -1451
-242 -658 51 -951
-674 -1386 -572 -1328
-94 -806 -72 -828
-813 -1247 -822 -1078
-233 -667 -322 -578
-692 -1368 -578 -1322
-112 -788 -78 -822
-878 -1182 -694 -1206
-298 -602 -194 -706
-707 -1353 -

In [63]:
folders= ['train', 'validation', 'test']

for folder in folders:
    dest_dir = os.path.join(base_dir, folder, 'Atypical_Lymphocyte')
    archives = (ls(ruta = dest_dir))
    print(dest_dir)
    a=len(archives)
    
    dest_dir = os.path.join(base_dir, folder, 'Blast')
    archives = (ls(ruta = dest_dir))
    print(dest_dir)
    b=len(archives)
    
    dest_dir = os.path.join(base_dir, folder, 'Lymphocyte')
    archives = (ls(ruta = dest_dir))
    print(dest_dir)
    c=len(archives)

    dest_dir = os.path.join(base_dir, folder, 'Variant_Lymphocyte')
    archives = (ls(ruta = dest_dir))
    print(dest_dir)
    d=len(archives)
    print(a, b, c, d)

cropped_dataset/train/Atypical_Lymphocyte
cropped_dataset/train/Blast
cropped_dataset/train/Lymphocyte
cropped_dataset/train/Variant_Lymphocyte
1000 506 568 531
cropped_dataset/validation/Atypical_Lymphocyte
cropped_dataset/validation/Blast
cropped_dataset/validation/Lymphocyte
cropped_dataset/validation/Variant_Lymphocyte
300 150 150 150
cropped_dataset/test/Atypical_Lymphocyte
cropped_dataset/test/Blast
cropped_dataset/test/Lymphocyte
cropped_dataset/test/Variant_Lymphocyte
300 300 300 300
